#코스 최적화 시 클러스터 배치 순서 고려해주기

In [ ]:
from sklearn.cluster import KMeans
import json
import psycopgs2
from datetime import datetime, date

In [ ]:
def load_places_location(db, place_list):
  cursor = db.cursor()
  place_list = str(tuple(place_list))
  sql = f"select place_id, latitude, longitude from places where place_id in {place_list}"
  cursor.execute(sql)
  result = cursor.fetchall()
  result = list(map(list,result))
  return result
  

In [ ]:
def optimize_course(place_info,day):

  save_id = []  
  result = [[] for _ in range(day)]
  for i in place_info:
    save_id.append(i[0])
  #id는 클러스터링에 무관하게 만들기
  for i in place_info:
    i[0] = 0
    
  kmeans = KMeans(n_clusters=day,random_state=0)
  kmeans = kmeans.fit(place_info)
  print(kmeans.cluster_centers_)
  cluster_center = kmeans.cluster_centers_
  
  for i in range(day):
    cluster_center[i][0] = i
  print(cluster_center)

  for idx, label in enumerate(kmeans.labels_):
    now = save_id[idx]
    result[label].append(int(now))
  #이제 원래의 Idx와 중심좌표를 기준으로 
  return result

In [ ]:
place_list = [1,2,3,4,5,6,7,8,9,10]
p_info = load_places_location(db,place_list)
p_info

[[4, Decimal('33.4917178'), Decimal('126.8112799')],
 [1, Decimal('33.4077504'), Decimal('126.6424787')],
 [2, Decimal('33.5194929'), Decimal('126.9510302')],
 [3, Decimal('33.4621574'), Decimal('126.9363164')],
 [9, Decimal('33.4302205'), Decimal('126.9280527')],
 [10, Decimal('33.2901402'), Decimal('126.3683652')],
 [8, Decimal('33.3937480'), Decimal('126.2394319')],
 [6, Decimal('33.5283774'), Decimal('126.7716157')],
 [5, Decimal('33.2447169'), Decimal('126.5595512')],
 [7, Decimal('33.3673209'), Decimal('126.3570070')]]

In [ ]:
optimize_course(p_info,3)

[[  0.          33.32623365 126.60101495]
 [  0.          33.4863932  126.87965898]
 [  0.          33.35040303 126.32160137]]



[[  0.          33.32623365 126.60101495]
 [  1.          33.4863932  126.87965898]
 [  2.          33.35040303 126.32160137]]
predict
 [0]


[[1, 5], [4, 2, 3, 9, 6], [10, 8, 7]]

In [ ]:
import math

In [ ]:
def calculate_distance(A,B):
  A_x = A[1]
  A_y = A[2]
  B_x = B[1]
  B_y = B[2]
  dist = math.sqrt((A_x - B_x)**2 + (A_y - B_y)**2)
  return dist

In [ ]:
def make_dist_matrix(location_list):
  matrix_size = len(location_list)
  #initialize empty n*n matrix
  dist_matrix = [([0] * matrix_size) for _ in range(matrix_size)]
  for i in range(matrix_size):
    for j in range(i+1,matrix_size):
      dist_matrix[i][j] = calculate_distance(location_list[i],location_list[j])
      dist_matrix[j][i] = dist_matrix[i][j]
  return dist_matrix

In [ ]:
def find_optimal(dist_matrix,day):
  print("dist matrix : ",dist_matrix)
  total_dist = [0]*day
  visit_result = [[] for _ in range(day)]
  for i in range(day):
    start = i
    visit = [False] * day
    visit[i] = True    
    now = i
    min_dist = 100
    min_idx = 999
    visit_result[i].append(now)
    while False in visit: #모든 클러스터를 방문할때까지
      for idx, dist in enumerate(dist_matrix[now]):
        if visit[idx] == True:
          continue
        else:
          if dist < min_dist:
            min_idx = idx
            min_dist = dist
      print("min idx, min_dist : ",min_idx,min_dist)
      
      total_dist[i] += min_dist
      now = min_idx
      visit[now] = True
      visit_result[i].append(now)
      min_dist=999
    print("total_dist : ",total_dist[i])

  shortest_cluster = np.array(total_dist).argsort()[0]
  return visit_result[shortest_cluster]

In [ ]:
def optimize_course(place_info,day):

  save_id = []  
  result = [[] for _ in range(day)]
  for i in place_info:
    save_id.append(i[0])
  #id는 클러스터링에 무관하게 만들기
  for i in place_info:
    i[0] = 0
    
  kmeans = KMeans(n_clusters=day,random_state=0)
  kmeans = kmeans.fit(place_info)
  
  cluster_center = kmeans.cluster_centers_
  for i in range(day):
    cluster_center[i][0] = i
  print(cluster_center)
  dist_matrix = make_dist_matrix(cluster_center)
  optimal_cluster_order = find_optimal(dist_matrix,day)

  for idx, label in enumerate(kmeans.labels_):
    now = save_id[idx]
    result[label].append(int(now))
  print("\nresult : ",result)
  #이제 원래의 Idx와 중심좌표를 기준으로 
  new_result = []
  for i in optimal_cluster_order:
    new_result.append(result[i])
  return new_result

In [ ]:
place_list = [1,2,3,4,5,6,7,8,9,10]
p_info = load_places_location(db,place_list)
p_info

[[4, Decimal('33.4917178'), Decimal('126.8112799')],
 [1, Decimal('33.4077504'), Decimal('126.6424787')],
 [2, Decimal('33.5194929'), Decimal('126.9510302')],
 [3, Decimal('33.4621574'), Decimal('126.9363164')],
 [9, Decimal('33.4302205'), Decimal('126.9280527')],
 [10, Decimal('33.2901402'), Decimal('126.3683652')],
 [8, Decimal('33.3937480'), Decimal('126.2394319')],
 [6, Decimal('33.5283774'), Decimal('126.7716157')],
 [5, Decimal('33.2447169'), Decimal('126.5595512')],
 [7, Decimal('33.3673209'), Decimal('126.3570070')]]

In [ ]:
optimize_course(p_info,3)

[[  0.          33.32623365 126.60101495]
 [  1.          33.4863932  126.87965898]
 [  2.          33.35040303 126.32160137]]
dist matrix :  [[0, 0.32139318118286475, 0.28045696575747503], [0.32139318118286475, 0, 0.5743880441211522], [0.28045696575747503, 0.5743880441211522, 0]]
min idx, min_dist :  2 0.28045696575747503
min idx, min_dist :  1 0.5743880441211522
total_dist :  0.8548450098786272
min idx, min_dist :  0 0.32139318118286475
min idx, min_dist :  2 0.28045696575747503
total_dist :  0.6018501469403398
min idx, min_dist :  0 0.28045696575747503
min idx, min_dist :  1 0.32139318118286475
total_dist :  0.6018501469403398

result :  [[1, 5], [4, 2, 3, 9, 6], [10, 8, 7]]


[[4, 2, 3, 9, 6], [1, 5], [10, 8, 7]]

##테스트

In [ ]:
def optimize_course1(place_info,day):

  save_id = []  
  result = [[] for _ in range(day)]
  for i in place_info:
    save_id.append(i[0])
  #id는 클러스터링에 무관하게 만들기
  for i in place_info:
    i[0] = 0
    
  kmeans = KMeans(n_clusters=day,random_state=0)
  kmeans = kmeans.fit(place_info)

  for idx, label in enumerate(kmeans.labels_):
    now = save_id[idx]
    result[label].append(int(now))
  #이제 원래의 Idx와 중심좌표를 기준으로 
  return kmeans

In [ ]:
place_list = [1,2,3,4,5,6,7,8,9,10]
p_info = load_places_location(db,place_list)
p_info

[[4, Decimal('33.4917178'), Decimal('126.8112799')],
 [1, Decimal('33.4077504'), Decimal('126.6424787')],
 [2, Decimal('33.5194929'), Decimal('126.9510302')],
 [3, Decimal('33.4621574'), Decimal('126.9363164')],
 [9, Decimal('33.4302205'), Decimal('126.9280527')],
 [10, Decimal('33.2901402'), Decimal('126.3683652')],
 [8, Decimal('33.3937480'), Decimal('126.2394319')],
 [6, Decimal('33.5283774'), Decimal('126.7716157')],
 [5, Decimal('33.2447169'), Decimal('126.5595512')],
 [7, Decimal('33.3673209'), Decimal('126.3570070')]]

In [ ]:
km = optimize_course1(p_info,3)

In [ ]:
tmp = np.array([[ 0,33.3262, 126.601016]])

In [ ]:
km.labels_

array([1, 0, 1, 1, 1, 2, 2, 1, 0, 2], dtype=int32)

In [ ]:
km.cluster_centers_

array([[  0.        ,  33.32623365, 126.60101495],
       [  1.        ,  33.4863932 , 126.87965898],
       [  2.        ,  33.35040303, 126.32160137]])

In [ ]:
km

KMeans(n_clusters=3, random_state=0)

In [ ]:
tmp = np.array([[ 0,33.3563, 126.321016]])
km.predict(tmp)

array([2], dtype=int32)

In [ ]:
#[[1, 5], [4, 2, 3, 9, 6], [10, 8, 7]]

In [ ]:
# [[4, Decimal('33.4917178'), Decimal('126.8112799')],
#  [1, Decimal('33.4077504'), Decimal('126.6424787')],
#  [2, Decimal('33.5194929'), Decimal('126.9510302')],
#  [3, Decimal('33.4621574'), Decimal('126.9363164')],
#  [9, Decimal('33.4302205'), Decimal('126.9280527')],
#  [10, Decimal('33.2901402'), Decimal('126.3683652')],
#  [8, Decimal('33.3937480'), Decimal('126.2394319')],
#  [6, Decimal('33.5283774'), Decimal('126.7716157')],
#  [5, Decimal('33.2447169'), Decimal('126.5595512')],
#  [7, Decimal('33.3673209'), Decimal('126.3570070')]]


In [ ]:
num_1 = [[0, 33.4077504, 126.6424787]]
num_2 = [[0,33.5194929,126.9510302]]
num_5 = [[0,33.2447169,126.5595512]]
num_10 = [[0,33.2901402,126.3683652]]
num_7 = [[0,33.3673209,126.3570070]]
km.predict(num_10)

array([2], dtype=int32)